In [2]:
# TODO: handle tilting of the effect with respect to head
# TODO: finding the angle of head
# TODO: y-coordinate from top-right corner, x-coordinate from top-left corner

In [3]:
import cv2
import numpy as np
import mediapipe as mp

In [4]:
mp_face_mesh = mp.solutions.face_mesh

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    static_image_mode=True,
    refine_landmarks=True,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

In [5]:
def get_landmarks(image):
    landmarks = {}
    h,w = image.shape[:2]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_mesh_result = face_mesh.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    if face_mesh_result.multi_face_landmarks:
        for i, landmark in enumerate(face_mesh_result.multi_face_landmarks[0].landmark): 
            x = landmark.x
            y = landmark.y
            relative_x = int(x * w)
            relative_y = int(y * h)
            landmarks[i+1] = (relative_x, relative_y)
    return landmarks


In [6]:
def get_coordinates(landmarks):
    coordinates = {
        "eye_left": [landmarks[30], [landmarks[158][0], landmarks[145][1]]],
        "eye_right": [landmarks[287], [landmarks[260][0], landmarks[381][1]]],
        # "shade": [landmarks[71], (landmarks[301][0], landmarks[301][1] + (landmarks[119][1] - landmarks[301][1]))]
        "shade": [(landmarks[140][0], landmarks[72][1]), landmarks[117], landmarks[346], landmarks[390]]
    }
    return coordinates

In [7]:
def add_effect(image, effect, icon_path, cordinates):
    item = cv2.imread(icon_path)
    pt1, pt2 = cordinates[effect]
    x, y, x_w, y_h = pt1[0], pt1[1], pt2[0], pt2[1]
    cropped = image[y:y_h, x:x_w, :]
    h, w, _ = cropped.shape
    item = cv2.resize(item, (w, h))
    blend = cv2.addWeighted(cropped, 0, item, 1.0, 0)
    
    return blend, x, y, x_w, y_h

In [8]:
def remove_whitespace(image, blend, x, y, threshold=225):
    for i in range(blend.shape[0]):
        for j in range(blend.shape[1]):
            for k in range(3):
                if blend[i][j][k] > threshold:
                    blend[i][j][k] = image[i + y][j + x][k]

In [9]:
def draw_iris_effects(image, coordinates):
    icon_path = "effects\\eye.png"
    for effect in ['eye_left', 'eye_right']:
        blend, x, y, x_w, y_h = add_effect(image, effect, icon_path, coordinates)
        remove_whitespace(
            image=image,
            blend=blend,
            x=x,
            y=y
        )
        image[y:y_h, x:x_w, :] = blend

In [10]:

def draw_specs_effect(image, coordinates):
    icon_path = "effects\\spec2.png"
    blend, x, y, x_w, y_h = add_effect(image, "shade", icon_path, coordinates)
    remove_whitespace(
        image=image,
        blend=blend,
        x=x,
        y=y
    )
    image[y:y_h, x:x_w, :] = blend
    

In [11]:
def draw_coordinates(image, points, color = [150, 0, 200]):
    neighbor_vector = [(0,0), (1,0), (0,1), (-1,0), (0, -1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
    for pt in points:
        image[pt[1]][pt[0]] = color
    return image

In [24]:
import math
def get_angle(coordinates):
    height = (coordinates[1][1] - coordinates[2][1])
    base = (coordinates[1][0] - coordinates[2][0])
    
    angle = math.atan(height/base) * 180/math.pi
    return angle
    

In [25]:
def get_rotated_image(im, angle):
    imHeight, imWidth = im.shape[0], im.shape[1]
    centreX, centreY = imWidth//2, imHeight//2
        
    rotationMat = cv2.getRotationMatrix2D(
    center=(centreX, centreY),
    angle=20,
    scale=1
    )
    
    cos = np.abs(rotationMat[0][0])
    sin = np.abs(rotationMat[1][0])
    
    newWidth = int((imHeight * sin) + (imWidth * cos))
    newHeight = int((imHeight * cos) + (imWidth * sin))
    
    rotationMat[0][2] = 0
    rotationMat[1][2] += newHeight/2 - centreY
    
    dst_mat = np.zeros((newHeight, newWidth, 4), np.uint8)
    rotatedMat = cv2.warpAffine(
        im,
        rotationMat,
        (newWidth, newHeight),
        dst_mat
    )
    
    return dst_mat, (newHeight, newWidth)

In [12]:
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    
    if ret:
        height, width, _ = frame.shape
        image = cv2.resize(frame, (width//2, height//2))[:,::-1]
        landmarks = get_landmarks(image=image)
        coordinates = get_coordinates(landmarks=landmarks)
        # draw_iris_effects(image, coordinates)
        draw_specs_effect(image, coordinates)
        image = draw_coordinates(image, coordinates['shade'], color=[255, 255, 255])
        
        cv2.imshow("Live face effects", image)
        if cv2.waitKey(5) & 0xFF == 27:
            cv2.destroyAllWindows() 
            cam.release()
            break

In [19]:
frame = cv2.imread("img.jpg")

cam = cv2.VideoCapture(0)
ret, frame = cam.read()


height, width, _ = frame.shape
# image = cv2.resize(frame, (width//2, height//2))[:,::-1]

In [20]:
landmarks = get_landmarks(image=frame)
coordinates = get_coordinates(landmarks=landmarks)

In [21]:
# for i in range(1, 469):
#     _img = draw_coordinates(image.copy(), list(landmarks.values()))
#     _img = draw_coordinates(_img, [landmarks[i]], [255, 255, 255])

#     cv2.imshow('Live face effects', image)
#     if cv2.waitKey(0) == 27:
#         cv2.destroyAllWindows()

In [22]:
image = draw_coordinates(frame, list(landmarks.values()))
image = draw_coordinates(
    image=image,
    points=[(landmarks[140][0], landmarks[72][1]), landmarks[117], landmarks[346], landmarks[390]],
    color=[255, 255, 255]
)

cv2.imshow('Live face effects', image)
if cv2.waitKey(0) == 27:
    cv2.destroyAllWindows()

In [23]:
cam.release()